# Лабораторная 2

In [2]:
from abc import ABCMeta, abstractmethod
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data\creditcard.csv")

In [4]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
class Classificator():
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def train(X, Y):
        '''Обучить модель'''
    
    @abstractmethod
    def predict(self, X):
        '''Предсказать данные'''

In [34]:
class DecisionTree(Classificator):
    model = None
    
    def train(self, X, Y, prediction=None, max_depth=None, leafValue=None):
        entropy = self.calculate_entropy(Y)
        igs = {}
        for i in len(X):
            for predicate in X.loc[i]:
                left = X.loc[i, X.loc[i] <= predicate]
                right = X.loc[i, X.loc[i] > predicate]
                entropy_left = calculate_entropy(Y_left)
                entropy_right = calculate_entropy(Y_right)
                entropies = {entropy_left + entropy_right}
                information_gain()
        return entropy
    
    def predict(self, X):
        pass
    
    def calculate_entropy(self, Y):
        count = len(df.Class)
        entropy = 0
        for count_class in df.Class.value_counts():
            p = count_class / count
            entropy -= p * np.log2(p)
        return entropy
    
    
    def information_gain(self, Y, entropy0, entropies):
        N = len(Y)
        sum_entropies = 0
        for i in Y.value_counts().index:
            Ni = entropies[i][0]
            Si = entropies[i][1]
            sum_entropies += Ni / N * Si
        return entropy0 - sum_entropies

In [7]:
class RandomForest(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [8]:
class GradientBoosting(Classificator):
    model = None
    
    def __init__():
        pass
    
    def train(X, Y):
        pass
    
    def predict(self, X):
        pass

In [27]:
tree = DecisionTree()
tree.train(df[:-1], df[-1:])

0.018343407709838624

In [42]:
len(df)

284807

In [33]:
for i in df.Class.value_counts().index:
    print(i)

0
1
